# Magnetostatic examples (3D)


## 1) Equations
Let us consider the following Maxwell's equations on a 3D domain $\Omega$:

$$
\left \{
\begin{array}{ll}
\underline{\text{curl}}~ \underline{h} = \underline{j} & \text{(Maxwell-Ampère),} \\
\text{div}~ \underline{b}  = 0 & \text{(Maxwell-Thomson),} 
\end{array} \right.
$$

where $\underline{h}$ is the magnetic field ($A/m$), $\underline{j}$ the current density ($A/m^2$) and $\underline{b}$ the flux density ($T$). The Maxwell-Thomson equation can be strongly ensured by defining a vector potential $\underline{a}$, such that 

$$ \underline{b} = \underline{\text{curl}}~\underline{a}. $$

Assuming all materials to be linear, isotropic and non-polarized, the behavior law reads

$$ \underline{h} = \nu ~ \underline{b}, $$

with $\nu = \mu^{-1}$ as the scalar magnetic reluctivity. By injecting these two relations into Maxwell-Ampère equations, one obtains the following primal (or B-conform) magnetostatic equation:  
 
 $$\underline{\text{curl}}~ (\nu ~ \underline{\text{curl}}~\underline{a}) = \underline{j}.$$
 
Taking any test function $\underline{a}^*$ in a certain space $H_{\text{curl}}(\Omega) = \{ a\in L^2(\Omega), \text{curl}~a \in L^2(\Omega) \}$ where the curl operator can be applied. The weak form of this equation reads

 $$ \forall \underline{a}^*\in H_{\text{curl}}(\Omega), \quad \int_\Omega \underline{a}^* \cdot \underline{\text{curl}}~ (\nu ~ \underline{\text{curl}}~\underline{a}) = \int_\Omega \underline{a}^* \cdot \underline{j}$$
 
 By integrating by part, *i.e.* by using:
  - first the Leibniz formula $ \underline{A} \cdot \underline{\text{curl}}~ \underline{B} = \underline{B} \cdot \underline{\text{curl}}~ \underline{A} - \text{div}~(\underline{A} \times \underline{B}) $,
  - then the Green-Ostrogradski formula $\int_\Omega \text{div}~(\underline{A} \times \underline{B}) =  \int_{\partial \Omega} (\underline{A} \times \underline{B}) \cdot \underline{n}$,
 
we obtain after some manipulations:
 $$  \forall \underline{a}^*\in H_{\text{curl}}(\Omega), \quad\int_\Omega \underline{\text{curl}}~\underline{a}^* \cdot (\nu~\underline{\text{curl}}~\underline{a}) - \int_{\partial \Omega} \underline{a}^* \cdot ((\nu~\underline{\text{curl}}~\underline{a} )\times \underline{n}) = \int_\Omega \underline{a}^* \cdot \underline{j}$$
 
Assuming the boundary term equal to 0, the weak form reads:

 $$ \boxed{  \forall \underline{a}^*\in H_{\text{curl}}(\Omega), \quad \int_\Omega \underline{\text{curl}}~\underline{a}^* \cdot (\nu~\underline{\text{curl}}~\underline{a}) = \int_\Omega \underline{a}^* \cdot \underline{j}}$$

Even with appropriate boundary conditions, the solution $a$ of this equation is not unique : indeed, for any $\psi$ such as $\nabla \psi=0$ on $\partial \Omega$, $a+\nabla \psi$ is still a solution. Therefore, there are three possibilities to solve the problem:
1. Add a small **penalized mass term**, so that the problem is not singular anymore but is (a little bit) perturbed
2. Solve the problem with an **iterative method**, where $\psi$ is fixed from the initial guess ; however, the right hand side should remain compatible
3. Add a **constraint (gauging)**, so that the problem is not singular anymore. 

In this example a 3D inductance is simulated.

## 2) Geometry

In [1]:
from ngcotree import *
from geometries import *
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo
from ngsolve.krylovspace import CGSolver

mesh = Mesh(inductance().GenerateMesh(maxh=0.002))
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

## 3) Current generation
### 3a) From a scalar potential with Neumann BC


In [2]:
sigma  = mesh.MaterialCF({"Coil" : 5.9e7}, default = 0)
I=1

def solveJgrad(mesh, I):
    fes = H1(mesh, definedon="Coil", dirichlet = "currentOutput")
    Jref = I/Integrate(CF(1)*ds("currentOutput"), mesh)

    # Weak form
    u, ustar = fes.TnT()
    Ku = BilinearForm(grad(ustar)*sigma*grad(u)*dx)
    fu = LinearForm(-ustar*Jref*ds("currentInput")) 

    # Assembly
    with TaskManager():
        Ku.Assemble()
        fu.Assemble()
    
    # Solving
    maxres = 1e-8
    maxit = 1000
                              
    Usol = GridFunction(fes)
    print("Solving...")
    with TaskManager():
        iterativeSolver = CGSolver(Ku.mat, freedofs=fes.FreeDofs(), atol   = maxres,  maxiter   = maxit)
        Usol.vec.data = iterativeSolver * fu.vec

    if len(iterativeSolver.residuals)==maxit:
        print("... Failure!")
    else:
        print("... Success!")
    print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")
    
    return Usol

U = solveJgrad(mesh, I)   
Draw(-sigma*grad(U), mesh, vectors = { "grid_size":50}, clipping = {"x" : 0, "y" : -1, "z" : 0, "dist" : 0})

Solving...
... Success!
Number of iterations = 93/1000 | Residual = 7.358452390119383e-09


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

### 3b) From a vector potential with Dirichlet BC

In [3]:
def solveJrot(mesh, I):
    fes = HCurl(mesh, definedon="Coil", dirichlet = "botCoil|topCoil|outCoil|intCoil")
       
    gfuDirichlet = GridFunction(fes)
    g = CF([(0,-I/(0.5*0.0254),0) if bc=="intCoil"  else (0,0,0) for bc in mesh.GetBoundaries()])
    gfuDirichlet.Set(g,BND)
    
    # Weak form
    T, Tstar = fes.TnT()
    Kt = BilinearForm(curl(Tstar)*curl(T)*dx)
    
    # Assembly
    with TaskManager():
        Kt.Assemble()
    
    r = - Kt.mat * gfuDirichlet.vec
    
    # Solving
    maxres = 1e-8
    maxit = 1000
                              
    Tsol = GridFunction(fes)
    print("Solving...")
    with TaskManager():
        iterativeSolver = CGSolver(Kt.mat, freedofs=fes.FreeDofs(), atol  = maxres,  maxiter   = maxit)
        Tsol.vec.data = iterativeSolver * r + gfuDirichlet.vec
        
    print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")
    
    return Tsol

T = solveJrot(mesh, I) 
Draw(curl(T), mesh, vectors = { "grid_size":50}, clipping = {"x" : 0, "y" : -1, "z" : 0, "dist" : 0})

Solving...
Number of iterations = 125/1000 | Residual = 7.397024608618317e-09


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

# 4) Resolutions
## 4a) Penalization

In [4]:
nu =  mesh.MaterialCF({"Bar" : 1/(2500*4e-7*np.pi), "Core" : 1/(1000*4e-7*np.pi)}, default = 1/(4e-7*np.pi))

def SolvePenal(mesh, nu, j , epsilon = 1e-6, order = 0):
    fes = HCurl(mesh, order = order, dirichlet = "out|antiSymmetry|currentInput|currentOutput")
    a, aStar = fes.TnT()
    
    bf = BilinearForm( curl(aStar) * (nu * curl(a)) * dx)
    bf += epsilon * aStar * nu * a * dx  # Penalization
    lf = LinearForm( aStar * j * dx )
    c = Preconditioner(bf, 'local')
    bf.Assemble()
    lf.Assemble()
    
    fd=fes.FreeDofs()
    fd[0] = 0
    
    aSol = GridFunction(fes)
    with TaskManager():
        iterativeSolver = CGSolver(bf.mat, c.mat, atol  = 1e-8,  maxiter   = 10000)
        aSol.vec.data = iterativeSolver * lf.vec
    print(f"Number of iterations = {iterativeSolver.iterations}/{10000} | Residual = {iterativeSolver.residuals[-1]}")
    return aSol

aPen = SolvePenal(mesh, nu, curl(T))
Draw(curl(aPen), mesh, vectors = { "grid_size":50}, clipping = {"x" : 0, "y" : 0, "z" : 1, "dist" : 0.01})

Number of iterations = 168/10000 | Residual = 9.758454637769928e-09


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

## 4b) Compatible RHS
Projection of the current on Hcurl in the whole domain.

In [5]:
def projL2Hcurl(mesh,JL2):
    W1 = HCurl(mesh,dirichlet = "out")
     ## Weak form
    T, Tstar = W1.TnT()
    Kt = BilinearForm(curl(Tstar)*curl(T)*dx)
    ft = LinearForm(curl(Tstar)*JL2*dx) 
    pre = Preconditioner(Kt, "local")
    ## Assembly
    with TaskManager():
        Kt.Assemble()
        ft.Assemble()

    maxres = 1e-8
    maxit = 10000

    Tsol = GridFunction(W1)
    
    print("Solving...")
    with TaskManager():
        iterativeSolver = CGSolver(Kt.mat, pre, atol  = 1e-8,  maxiter   = 10000)
        Tsol.vec.data = iterativeSolver * ft.vec   
    print(f"Number of iterations = {iterativeSolver.iterations}/{10000} | Residual = {iterativeSolver.residuals[-1]}")
    return Tsol

Tsol = projL2Hcurl(mesh,curl(T))
#Tsol = projL2Hcurl(mesh,-sigma*grad(U))
Draw(curl(Tsol), mesh, vectors = { "grid_size":100}, clipping = {"x" : 0, "y" : -1, "z" : 0, "dist" : 0})

Solving...
Number of iterations = 245/10000 | Residual = 9.519899028234845e-09


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

Computation of 3D magnetostatics with compatible RHS and no gauging.

In [6]:
def solveCompatibleRHS(mesh,Tsol):
    jHCurl = curl(Tsol)
    W1 = HCurl(mesh,dirichlet = "out|antiSymmetry|currentInput|currentOutput")         

    ## Weak form
    a, astar = W1.TnT()
    Kmag_singular = BilinearForm(nu*curl(astar)*curl(a)*dx)
    fmag_singular = LinearForm(curl(astar)*Tsol*dx) 
    #fmag_singular = LinearForm(astar*curl(Tsol)*dx) 

    ## Assembly
    with TaskManager():
        Pre = Preconditioner(Kmag_singular, "local") 
        Kmag_singular.Assemble()
        fmag_singular.Assemble()

    maxres = 1e-8
    maxit = 10000

    Asol = GridFunction(W1)
    
    print("Solving...")
    with TaskManager():
        iterativeSolver = CGSolver(Kmag_singular.mat, Pre, atol  = maxres,  maxiter   = maxit)
        #iterativeSolver = CGSolver(Kmag_singular.mat, freedofs = W1.FreeDofs(), tol  = maxres,  maxiter  = maxit)
        Asol.vec.data = iterativeSolver * fmag_singular.vec   

    print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")
    return Asol, iterativeSolver.residuals

Arhs, info = solveCompatibleRHS(mesh,Tsol) # compatible current
Draw(curl(Arhs), mesh, vectors = { "grid_size":50}, clipping = {"x" : 0, "y" : 0, "z" : 1, "dist" : 0.01})

Solving...
Number of iterations = 168/10000 | Residual = 9.758502889405522e-09


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

## 4c) Tree-Cotree gauging

The system is smaller and converges even with non-compatible RHS, but the condition number is increased so the CG needs more iterations to converge.

In [7]:
def SolveCoTree(mesh, nu, j , epsilon = 1e-6, order = 0):
    fes = HCurl(mesh, order = order, dirichlet = "out|antiSymmetry|currentInput|currentOutput")
    fes.FreeDofs()[:] = CoTreeBitArray(mesh, fes)
    
    a, aStar = fes.TnT()
    bf = BilinearForm( curl(aStar) * (nu * curl(a)) * dx)
    lf = LinearForm( aStar * j * dx )
    c = Preconditioner(bf, 'local')
    bf.Assemble()
    lf.Assemble()
    
    fd=fes.FreeDofs()
    fd[0] = 0
    
    aSol = GridFunction(fes)
    with TaskManager():
        iterativeSolver = CGSolver(bf.mat, c.mat, atol  = 1e-8,  maxiter   = 10000)
        aSol.vec.data = iterativeSolver * lf.vec
    print(f"Number of iterations = {iterativeSolver.iterations}/{10000} | Residual = {iterativeSolver.residuals[-1]}")
    return aSol

aPen = SolveCoTree(mesh, nu, -sigma*grad(U)) # non-compatible current
Draw(curl(aPen), mesh, vectors = { "grid_size":50}, clipping = {"x" : 0, "y" : 0, "z" : 1, "dist" : 0.01})

Number of iterations = 830/10000 | Residual = 9.701335932034209e-09


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene